## Question answering. Практическое задание (PJ) Граур Андрей Константинович

Для закрепления материала модуля предлагаем вам решить задачу QA для датасета [SberQuad](https://huggingface.co/datasets/sberquad), используя любые доступные вам средства.

Для достижения наилучшего результата уделите внимание подбору гиперапарметров как в плане архитектуры, так и в плане обучения модели.

Критерии оценивания проекта:

- общее качество кода и следование PEP-8;
- использование рекуррентных сетей;
- использованы варианты архитектур, близкие к state of the art для данной задачи;
- произведен подбор гиперпараметров;
- использованы техники изменения learning rate (lr scheduler);
- использована адекватная задаче функция потерь;
- использованы техники регуляризации;
- корректно проведена валидация модели;
- использованы техники ensemble;
- использованы дополнительные данные;
- итоговое значение метрики качества > 0.75 (f1).

#### Хотел бы подчернуть, что я не стал использовать для выполнения этого задания Google Collab, так как обучение модели занимает довольно много времени, а бесплатное использование GPU на этой платформе сильно ограничено. Поэтому я запускал код на своей локальной среде, с видеокартой NVIDIA GeForce GTX 1660 с поддержкой CUDA. Для этого было установлены все необходимые библиотеки и настроено окружение в Anakonda.

## 1) Импортирую необхоимдые мне зависимости

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [2]:
import transformers
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
import warnings
warnings.simplefilter("ignore")

## 2) Достаю данные

In [3]:
from datasets import load_dataset
dataset = load_dataset("sberquad")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to C:/Users/Andrew/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/3e53185d0662a022bd749ec2b67b20499070efcbc1475428b0dad76c2cf8b06b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

## 3) Проверяю данные и стараюсь их провалидировать и удалить лишнее

In [4]:
print('Train Data Sample.....')
train_data = dataset["train"]
for data in train_data:
    print()
    print('ID -' + data['id'])
    print('TITLE - ' + data['title'])
    print('CONTEXT - ' + data['context'])
    print('ANSWERS - ' + data['answers']['text'])
    print('ANSWERS START INDEX - ' + data['answers']['answer_start'])
    print()
    break

print('---'*30)
print(bold + 'Validation Data Sample.....' + bold)
train_data = dataset["validation"]
for data in train_data:
    print()
    print('ID -' + data['id'])
    print('TITLE - '+ data['title'])
    print('CONTEXT - '+ data['context'])
    print('ANSWERS - ' + data['answers']['text'])
    print('ANSWERS START INDEX - ' + data['answers']['answer_start'])
    print()
    break



Train Data Sample.....
 
ID - 62310
TITLE -  SberChallenge
CONTEXT -  В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
ANSWERS -  ['известковыми выделениями сине-зелёных водорослей']
ANSWERS START INDEX -  [109]
 
------------------------------------------------------------------------------------------
Validation Data Sample.....
 
ID - 60544
TITLE -  SberChallenge
CONTEXT -  Первые упоминания о строении человечес

In [7]:
dataset["validation"][777] , dataset["test"][777]

({'id': 76422,
  'title': 'SberChallenge',
  'context': 'Относительно вхождения уральской семьи языков в более крупные генетические объединения существуют разные гипотезы, ни одна из которых не признана специалистами по уральским языкам. Согласно ностратической гипотезе, уральская семья, наряду с другими языковыми семьями и макросемьями входит в состав более крупного образования — ностратической макросемьи, причём сближается там с юкагирскими языками, образуя уральско-юкагирскую группу. Данная позиция, однако, была подвергнута критике различными специалистами, считается весьма спорной и её выводы не принимаются многими компаративистами, которые рассматривают теорию ностратических языков либо как, в худшем случае, полностью ошибочную или как, в лучшем случае, просто неубедительную[8][9]. Примерно до середины 1950-х гг. была популярна урало-алтайская гипотеза, объединявшая в одну макросемью уральские и алтайские языки, однако в настоящее время она не пользуется поддержкой лингвистов. С у

In [8]:
ids = np.random.choice(range(len(dataset["validation"])), int(len(dataset["validation"])*0.1*2.2), replace=False)
len(ids)

1107

In [9]:
choice = np.random.choice(range(len(ids)), int(len(ids)*0.2), replace=False)
test_mask = np.zeros(ids.shape[0], dtype=bool)
test_mask[choice] = True

test_ids = set_ids[test_mask]
val_ids = set_ids[~test_mask]
len(val_ids), len(test_ids)

(886, 221)

## 4) Формирую датасет и произвожу препроцессинг

In [10]:

from datasets import Dataset, DatasetDict


DATASET = DatasetDict({
    'train': dataset["train"].select(
            np.random.choice(range(len(dataset["train"])), int(len(dataset["train"])*part_of_data), replace=False)
        ),
    'validation': dataset["validation"].select(val_ids
        ),
    'test': dataset["validation"].select(test_ids
        )
})

DATASET

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4532
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 886
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 221
    })
})

### 4.1) Выполняю токенизацию вопроса и контекста с использованием предобученной модели и вывожу результаты на экран:

1. Импортируется класс `AutoTokenizer` из модуля `transformers`.
2. Задается переменная `trained_checkpoint` с указанием пути к предобученной модели.
3. Создается экземпляр класса `AutoTokenizer` с помощью переменной `custom_tokenizer`, используя предобученную модель.
4. Значения переменных `custom_context`, `custom_question` и `custom_answer` присваиваются из соответствующих элементов в `DATASET`.
5. Происходит токенизация вопроса и контекста с помощью метода `custom_tokenizer()`, с заданными параметрами для максимальной длины, обрезки, шага и возврата лишних токенов.
6. Выводится количество признаков, полученных из 4 примеров, и информация о распределении этих признаков.
7. Выводится вопрос, контекст и ответ.
8. В цикле происходит итерация по `input_ids` и декодирование каждого контекстного куска.
9. Декодированный контекстный кусок выводится на экран.

In [11]:
from transformers import AutoTokenizer

trained_checkpoint = "timpal0l/mdeberta-v3-base-squad2"
custom_tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
custom_context = DATASET["train"][0]["context"]
custom_question = DATASET["train"][0]["question"]
custom_answer = DATASET["train"][0]["answers"]["text"]
custom_inputs = custom_tokenizer(
    custom_question,
    custom_context,
    max_length=160,
    truncation="only_second", 
    stride=70,  
    return_overflowing_tokens=True,  
)
print(f"The 4 examples gave {len(custom_inputs['input_ids'])} features.")
print(f"Here is where each comes from: {custom_inputs['overflow_to_sample_mapping']}.")
print('Question: ',custom_question)
print()
print('Context : ',custom_context)
print()
print('Answer: ', custom_answer)
print('-------------------------------------------------')
for i, ids in enumerate(custom_inputs["input_ids"]):
    print('Context piece', i+1)
    print(custom_tokenizer.decode(ids[ids.index(1):]))
    print()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The 4 examples gave 3 features.
Here is where each comes from: [0, 0, 0].
Question:  Куда впадает река Кефисс?
 
Context :  Афины расположены на центральной равнине Аттики, так называемом бассейне, который окружен горой Эгалео с запада, горой Парниc с севера, горой Пенделикон с северо-востока и горой Имитос с востока и омывается заливом Сароникос с юго-запада. Поскольку Афины заняли всю равнину, в дальнейшем им будет очень сложно продолжать рост территории из-за естественных границ. Тем не менее, постоянно расширяются пригороды на окраинах городах, и сегодня Палини, город на востоке Аттики, является восточной окраиной города, Айос-Стефанос (Άγιος Στέφανος) — северо-восточной окраиной, Ахарне — северной, Лиосия (Λιόσια) — северо-западной окраиной, Мосхато (Μοσχάτο) — западной и Варкиза (Βάρκιζα) — южная окраина Афин. Город разделен рекой Кефисс, которая стекает с массива Пенделикон — Парниc, впадает в фалерскую бухту залива Сароникос и отделяет Пирей от остальной части Афин.
 
Answer:  

Функция `train_data_preprocess` принимает на вход примеры данных и производит предобработку. Внутри функции определена вспомогательная функция `find_context_start_end_index`, которая находит начальный и конечный индексы контекста в последовательности токенов. Затем, внутри основной функции, происходит подготовка вопросов, контекста и ответов. С помощью токенизатора `custom_tokenizer` производится токенизация вопросов и контекста с заданными параметрами. Затем, в цикле, происходит обработка каждого примера данных. Извлекаются необходимые информации, такие как индексы начала и конца ответа, индексы начала и конца контекста. Затем, в зависимости от условия, добавляются соответствующие позиции начала и конца ответа в списки `start_positions` и `end_positions`. Наконец, возвращается обновленный набор данных. В конце кода, создается выборка `custom_train_sample` из общего набора данных, применяется функция `train_data_preprocess` к выборке и создается новый набор данных `custom_train_dataset`. Выводится длина исходного набора данных и длина нового набора данных.

In [12]:
def train_data_preprocess(examples):
    def find_context_start_end_index(sequence_ids):
        token_idx = 0
        while sequence_ids[token_idx] != 1:
            token_idx += 1
        context_start_idx = token_idx
        while sequence_ids[token_idx] == 1:
            token_idx += 1
        context_end_idx = token_idx - 1
        return context_start_idx, context_end_idx

    custom_questions = [q.strip() for q in examples["question"]]
    custom_context = examples["context"]
    custom_answers = examples["answers"]

    custom_inputs = custom_tokenizer(
        custom_questions,
        custom_context,
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    start_positions = []
    end_positions = []

    for i, mapping_idx_pairs in enumerate(custom_inputs['offset_mapping']):
        context_idx = custom_inputs['overflow_to_sample_mapping'][i]
        answer = custom_answers[context_idx]
        answer_start_char_idx = answer['answer_start'][0]
        answer_end_char_idx = answer_start_char_idx + len(answer['text'][0])
        tokens = custom_inputs['input_ids'][i]
        sequence_ids = custom_inputs.sequence_ids(i)
        context_start_idx, context_end_idx = find_context_start_end_index(sequence_ids)
        context_start_char_index = mapping_idx_pairs[context_start_idx][0]
        context_end_char_index = mapping_idx_pairs[context_end_idx][1]

        if (context_start_char_index > answer_start_char_idx) or (
                context_end_char_index < answer_end_char_idx):
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start_idx
            while idx <= context_end_idx and mapping_idx_pairs[idx][0] <= answer_start_char_idx:
                idx += 1
            start_positions.append(idx - 1)
            idx = context_end_idx
            while idx >= context_start_idx and mapping_idx_pairs[idx][1] > answer_end_char_idx:
                idx -= 1
            end_positions.append(idx + 1)

    custom_inputs["start_positions"] = start_positions
    custom_inputs["end_positions"] = end_positions

    return custom_inputs

custom_train_sample = custom_dataset["train"].select([i for i in range(200)])
custom_train_dataset = custom_train_sample.map(
    train_data_preprocess,
    batched=True,
    remove_columns=custom_dataset["train"].column_names
)

len(custom_dataset["train"]), len(custom_train_dataset)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

(4532, 200)

### 4.2) Увеличиваю количество после токенизации

Функция `print_context_and_answer` принимает индекс `idx` и, по умолчанию, работает с набором данных `custom_dataset["train"]`. Внутри функции происходит вывод информации о вопросе, контексте и ответе до и после токенизации. Сначала выводятся теоретические значения, такие как вопрос, контекст и ответ. Затем выводятся индексы начала и конца ответа. Затем, при помощи токенизатора `custom_tokenizer`, происходит токенизация и декодирование вопроса, контекста и ответа. Выводятся токенизированные значения вопроса, контекста и ответа, а также индексы начала и конца ответа в токенах.  Затем функция вызывается для четырех разных индексов, чтобы продемонстрировать вывод информации для разных примеров данных.

In [13]:
def print_context_and_answer(idx, mini_ds=custom_dataset["train"]):
    print(idx)
    print('----')

    custom_question = mini_ds[idx]['question']
    custom_context = mini_ds[idx]['context']
    custom_answer = mini_ds[idx]['answers']['text']

    print('Theoretical values:')
    print(' ')
    print('Question:')
    print(custom_question)
    print(' ')
    print('Context:')
    print(custom_context)
    print(' ')
    print('Answer:')
    print(custom_answer)
    print(' ')
    
    answer_start_char_idx = mini_ds[idx]['answers']['answer_start'][0]
    answer_end_char_idx = answer_start_char_idx + len(mini_ds[idx]['answers']['text'][0])
    print('Start and end index of text:', answer_start_char_idx, answer_end_char_idx)
    print('----' * 20)
    print('Values after tokenization:')
    
    sep_tok_index = custom_train_dataset[idx]['input_ids'].index(1)  # get index for [SEP]
    question_ = custom_train_dataset[idx]['input_ids'][:sep_tok_index + 1]
    question_decoded = custom_tokenizer.decode(question_)
    context_ = custom_train_dataset[idx]['input_ids'][sep_tok_index + 1:]
    context_decoded = custom_tokenizer.decode(context_)
    start_idx = custom_train_dataset[idx]['start_positions']
    end_idx = custom_train_dataset[idx]['end_positions']
    answer_toks = custom_train_dataset[idx]['input_ids'][start_idx:end_idx]
    answer_decoded = custom_tokenizer.decode(answer_toks)

    print(' ')
    print('Question:')
    print(question_decoded)
    print(' ')
    print('Context:')
    print(context_decoded)
    print(' ')
    print('Answer:')
    print(answer_decoded)
    print(' ')
    print('Start pos and end pos of tokens:', custom_train_dataset[idx]['start_positions'], custom_train_dataset[idx]['end_positions'])
    print('____' * 20)

print_context_and_answer(0)
print_context_and_answer(1)
print_context_and_answer(2)
print_context_and_answer(3)

0
----
Theoretical values :
 
Question: 
чем представлены органические остатки?
 
Context: 
В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
 
Answer: 
['известковыми выделениями сине-зелёных водорослей']
 
Start and end index of text:  109 157
--------------------------------------------------------------------------------
Values after tokenization:
 
Question: 
[CLS]
 
Context: 
Куда впадает река Кефисс?[SEP] Афи


### 4.2) Метрики для evaluation

In [15]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_map = inputs.pop("overflow_to_sample_mapping")
    base_ids = []
    for i in range(len(inputs["input_ids"])):
        base_context_idx = sample_map[i]
        base_ids.append(examples["id"][base_context_idx])
        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]
    inputs["base_id"] = base_ids
    return inputs
data_val_sample = DATASET["validation"].select([i for i in range(100)])
eval_set = data_val_sample.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=DATASET["validation"].column_names,
)
len(eval_set)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

100

## 5) Тюнинг

### 5.1) CUDA
Для обучения планирую использовать свою ПК с видеокартой NVIDIA GeForce GTX 1660 с поддержкой cuda. Для этого было установлены все необходимые библиотека и настроено окружение в Anakonda.

### 5.2) Эксперименты

Здесь происходит оценка модели вопрос-ответ. Сначала создается копия `eval_set`, из которой удаляются столбцы "base_id" и "offset_mapping". Затем формат данных устанавливается в формат "torch". Затем определяется устройство, на котором будет выполняться вычисления (GPU или CPU). Создается словарь `batch`, в котором данные из `eval_dataset` переносятся на устройство. Модель `model` инициализируется с использованием предобученного чекпоинта и переносится на устройство. Затем с помощью `torch.no_grad()` происходит вычисление вывода модели на данных `batch`. Полученные логиты для начала и конца ответа переводятся в массивы numpy. В конце выводятся размерности полученных массивов.

In [16]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_dataset = eval_set.remove_columns(["base_id", "offset_mapping"])
eval_dataset.set_format("torch")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_dataset[k].to(device) for k in eval_dataset.column_names}
model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

with torch.no_grad():
    output = model(**batch)

start_logits = output.start_logits.cpu().numpy()
end_logits = output.end_logits.cpu().numpy()

start_logits.shape, end_logits.shape

((100, 512), (100, 512))

Здесь определяется функция `predict_answers_and_evaluate`, которая принимает логиты начала и конца ответа, набор данных `eval_dataset` и примеры для оценки. Внутри функции создается словарь `example_to_features`, который связывает базовый идентификатор с соответствующими индексами в `eval_dataset`. Затем происходит итерация по примерам и их связанным функциям. Для каждого примера происходит итерация по функциям и вычисляются логиты начала и конца ответа, а также смещения. Затем происходит сортировка логитов и выбор лучших индексов начала и конца ответа. Для каждой комбинации начала и конца ответа проверяются условия и, если они выполняются, добавляется ответ в список. Если есть хотя бы один ответ, выбирается лучший на основе суммы логитов. Если ответов нет, добавляется пустой ответ. Затем происходит оценка предсказанных ответов с использованием метрики `squad` и возвращаются предсказанные ответы и метрика.

In [17]:
import numpy as np
import collections
import evaluate

def predict_answers_and_evaluate(start_logits, end_logits, eval_dataset, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(eval_dataset):
        example_to_features[feature["base_id"]].append(idx)
    n_best = 20
    max_answer_length = 30
    predicted_answers = []

    for example in examples:
        example_id = example["id"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = eval_dataset["offset_mapping"][feature_index]
            start_indexes = np.argsort(start_logit).tolist()[::-1][:n_best]
            end_indexes = np.argsort(end_logit).tolist()[::-1][:n_best]

            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    answers.append({
                        "text": context[offsets[start_index][0]: offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    })

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append({
                "id": str(example_id),
                "prediction_text": best_answer["text"]
            })
        else:
            predicted_answers.append({
                "id": str(example_id),
                "prediction_text": ""
            })

    metric = evaluate.load("squad")
    theoretical_answers = [
        {"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples
    ]
    metric_2 = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    return predicted_answers, metric_2

In [18]:
pred_answers, metrics_2 = predict_answers_and_evaluate(start_logits, end_logits,
                                                     eval_set, data_val_sample)
metrics_2

{'exact_match': 46.0, 'f1': 78.84236319236315}

Здесь определяется класс `DataQA`, который является подклассом `Dataset` из `torch.utils.data`. В конструкторе класса определяется режим работы (`train`, `validation` или `test`) и соответствующий набор данных. В зависимости от режима работы происходит выбор соответствующего набора данных и применение соответствующей предобработки данных. Метод `__len__` возвращает длину данных, а метод `__getitem__` возвращает элемент данных по индексу `idx`. Данные возвращаются в виде словаря, содержащего тензоры `input_ids` и `attention_mask`. Если режим работы - `train`, также возвращаются тензоры `start_positions` и `end_positions`.

In [19]:
from torch.utils.data import DataLoader, Dataset

class DataQA(Dataset):
    def __init__(self, dataset, mode="train"):
        self.mode = mode

        if self.mode == "train":
            self.dataset = dataset["train"]
            self.data = self.dataset.map(train_data_preprocess, batched=True, remove_columns=dataset["train"].column_names)
        elif self.mode == "validation":
            self.dataset = dataset["validation"]
            self.data = self.dataset.map(preprocess_validation_examples, batched=True, remove_columns=dataset["validation"].column_names)
        else:
            self.dataset = dataset["test"]
            self.data = self.dataset.map(preprocess_validation_examples, batched=True, remove_columns=dataset["test"].column_names)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out = {}
        example = self.data[idx]
        out['input_ids'] = torch.tensor(example['input_ids'])
        out['attention_mask'] = torch.tensor(example['attention_mask'])

        if self.mode == "train":
            out['start_positions'] = torch.unsqueeze(torch.tensor(example['start_positions']), dim=0)
            out['end_positions'] = torch.unsqueeze(torch.tensor(example['end_positions']), dim=0)

        return out

In [20]:
train_dataset = DataQA(DATASET, mode="train")
val_dataset = DataQA(DATASET, mode="validation")

for i, d in enumerate(train_dataset):
    for k in d.keys():
        print(k + ' : ', d[k].shape)
    print('--------------------------------------------------------------------------------')

    if i == 3:
        break

print('--------------------------------------------------------------------------------')

for i, d in enumerate(val_dataset):
    for k in d.keys():
        print(k + ' : ', len(d[k]))
    print('--------------------------------------------------------------------------------')

    if i == 3:
        break

Map:   0%|          | 0/4532 [00:00<?, ? examples/s]

Map:   0%|          | 0/886 [00:00<?, ? examples/s]

input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
input_ids :  torch.Size([512])
attention_mask :  torch.Size([512])
start_positions :  torch.Size([1])
end_positions :  torch.Size([1])
--------------------------------------------------------------------------------
____________________________________________________________________________________________________
input_ids :  512
attention_mask :  

### 5.3) Батчи

Здесь создаются загрузчики данных (`train_dataloader` и `eval_dataloader`) с использованием класса `DataLoader` из `torch.utils.data`. Для обучающего загрузчика (`train_dataloader`) указывается перемешивание данных (`shuffle=True`), функция сборки данных (`collate_fn=default_data_collator`) и размер пакета (`batch_size=2`). Для валидационного загрузчика (`eval_dataloader`) указывается только функция сборки данных (`collate_fn=default_data_collator`) и размер пакета (`batch_size=2`). Затем происходит итерация по пакетам в обучающем загрузчике и выводятся размерности каждого ключа в пакете. Затем происходит итерация по пакетам в валидационном загрузчике и выводятся размерности каждого ключа в пакете. 

In [21]:
from transformers import default_data_collator
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=2,
)
eval_dataloader = DataLoader(
    val_dataset, collate_fn=default_data_collator, batch_size=2
)

for batch in train_dataloader:
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    print(batch['start_positions'].shape)
    print(batch['end_positions'].shape)
    break

print('------------------------------------------------------------')

for batch in eval_dataloader:
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    break

torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 1])
torch.Size([2, 1])
------------------------------------------------------------
torch.Size([2, 512])
torch.Size([2, 512])


### 5.3) Определяем модель

Еще раз проверяем, что используем CUDA

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Available device: {device}')

model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)
model = model.to(device)

Available device: cuda


### 5.4) Тренирую модель

3 эпохи

In [23]:
from transformers import AdamW
from tqdm.notebook import tqdm
import datetime
import numpy as np
import collections
import evaluate

optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 3
total_steps = len(train_dataloader) * epochs
print(total_steps)


def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))



6876


In [24]:
validation_processed_dataset = DATASET["validation"].map(preprocess_validation_examples,
            batched=True,remove_columns = DATASET["validation"].column_names,
               )

Loading cached processed dataset at C:\Users\Andrew\.cache\huggingface\datasets\sberquad\sberquad\1.0.0\3e53185d0662a022bd749ec2b67b20499070efcbc1475428b0dad76c2cf8b06b\cache-44f3730357a550f0.arrow


In [26]:
import random
import time
import numpy as np

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
stats = []
total_train_time_start = time.time()

for epoch in range(epochs):
    print(f'Номер эпохи: {epoch}')
    t0 = time.time()

    training_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed_time = format_time(time.time() - t0)
            print('Прогресс: {:}'.format(elapsed_time))

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        model.zero_grad()

        result = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       start_positions=start_positions,
                       end_positions=end_positions,
                       return_dict=True)

        loss = result.loss
        training_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = training_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("Loss: {0:.2f}".format(avg_train_loss))
    print("Время обучения: {:}".format(training_time))

    print("Валидация")
    t0 = time.time()
    model.eval()
    start_logits, end_logits = [], []
    for step, batch in enumerate(eval_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        with torch.no_grad():
            result = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           return_dict=True)
        start_logits.append(result.start_logits.cpu().numpy())
        end_logits.append(result.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    answers, metrics_ = predict_answers_and_evaluate(start_logits,
                                                     end_logits,
                                                     validation_processed_dataset,
                                                     DATASET["validation"])
    print(f'Match: {metrics_["exact_match"]}')
    print(f'F1: {metrics_["f1"]}')
    validation_time = format_time(time.time() - t0)
    print("Время валидации: {:}".format(validation_time))

print("Полное время обучения {:} (h:mm:ss)".format(format_time(time.time() - total_train_time_start)))

Номер эпохи: 0
Прогресс: 0:11:47
Прогресс: 0:23:18
Прогресс: 0:34:50
Прогресс: 0:46:21
Прогресс: 0:58:05
Прогресс: 1:09:33
Прогресс: 1:21:06
Прогресс: 1:32:39
Прогресс: 1:43:55
Прогресс: 1:54:48
Прогресс: 2:05:42
Прогресс: 2:16:35
Прогресс: 2:27:29
Прогресс: 2:38:23
Прогресс: 2:49:16
Прогресс: 3:00:10
Прогресс: 3:11:03
Прогресс: 3:21:57
Прогресс: 3:32:50
Прогресс: 3:43:44
Прогресс: 3:54:37
Прогресс: 4:05:31
Прогресс: 4:16:24
Прогресс: 4:27:18
Прогресс: 4:38:12
Прогресс: 4:49:05
Прогресс: 5:00:01
Прогресс: 5:10:55
Прогресс: 5:21:48
Прогресс: 5:32:42
Прогресс: 5:43:35
Прогресс: 5:54:29
Прогресс: 6:05:22
Прогресс: 6:16:16
Прогресс: 6:27:09
Прогресс: 6:38:03
Прогресс: 6:48:56
Прогресс: 6:59:50
Прогресс: 7:10:43
Прогресс: 7:21:37
Прогресс: 7:32:30
Прогресс: 7:43:24
Прогресс: 7:54:17
Прогресс: 8:05:11
Прогресс: 8:16:04
Прогресс: 8:26:59
Прогресс: 8:37:52
Прогресс: 8:48:46
Прогресс: 8:59:39
Прогресс: 9:10:33
Прогресс: 9:21:26
Прогресс: 9:32:20
Прогресс: 9:43:13
Прогресс: 9:54:07
Прогресс: 10:

Полное время обучения заняло чуть больше одного дня и восьми часов.

### 5.5) Тестирую модель

In [28]:
test_dataset = DataQA(DATASET, mode="test")
test_dataloader = DataLoader(
    test_dataset, collate_fn=default_data_collator, batch_size=2
)
test_processed_dataset = DATASET["test"].map(preprocess_validation_examples,
                                             batched=True,
                                             remove_columns=DATASET["test"].column_names,
)
model.eval()
start_logits, end_logits = [], []
for step, batch in enumerate(test_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    with torch.no_grad():
        result = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       return_dict=True)

    start_logits.append(result.start_logits.cpu().numpy())
    end_logits.append(result.end_logits.cpu().numpy())
start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
answers, metrics_ = predict_answers_and_evaluate(start_logits,
                                                 end_logits,
                                                 test_processed_dataset,
                                                 DATASET["test"])
print(f'Метрики: {metrics_["exact_match"]}')
print(f'F1: {metrics_["f1"]}')

Loading cached processed dataset at C:\Users\Andrew\.cache\huggingface\datasets\sberquad\sberquad\1.0.0\3e53185d0662a022bd749ec2b67b20499070efcbc1475428b0dad76c2cf8b06b\cache-f377e91e9237f2df.arrow
Loading cached processed dataset at C:\Users\Andrew\.cache\huggingface\datasets\sberquad\sberquad\1.0.0\3e53185d0662a022bd749ec2b67b20499070efcbc1475428b0dad76c2cf8b06b\cache-f377e91e9237f2df.arrow


Метрики: 42.081447963800905
F1: 78.5412536710465


## 6) Вывод

Я решил задачу QA для датасета SberQuad и получил метрику **F1 = 78.5412536710465**